In [1]:
import pandas as pd
import tokenize_uk
from typing import List
from ast import literal_eval

In [2]:
def read_stop_words(file):
    with open(file) as f:
        stop_words = f.read().split('\n')

    return stop_words

In [3]:
def read_all_frames(files):
    frames= []
    for file in files:
        df = pd.read_csv(file)
        frames.append(df)
    
    return pd.concat(frames)

In [4]:
files = [
    'dataset/booking/booking-train.csv',
    'dataset/booking/booking-test.csv',
    'dataset/booking/booking-val.csv'
]

In [5]:
full_df = read_all_frames(files)

In [6]:
full_df

,title,pos_text,neg_text,ratingValue,bestRating,hotel,rating
0,Лише дівчата на рецепції - три рази мені мінял...,Лише дівчата на рецепції - три рази мені мінял...,"Все. Одного досвіду вистарчило, щоб більше сюд...",4.6,10.0,verhovina.uk.html,2
1,"Оформлення кімнати хороше, досить приємне, на ...","Оформлення кімнати хороше, досить приємне, на ...",Nan,9.2,10.0,verhovina.uk.html,5
2,"Усе відмінно, завдяки якісному сервісу ми завж...","Усе відмінно, завдяки якісному сервісу ми завж...",Рекомендую людям котрі подорожують власним тра...,10.0,10.0,verhovina.uk.html,5
3,Ціна/якість в принципі,Ціна/якість в принципі,"душова, трохи старий ремонт, але за 500 грн за...",7.5,10.0,verhovina.uk.html,4
4,"Приїхали з сином біля сьомої ранку, сонні та в...","Приїхали з сином біля сьомої ранку, сонні та в...",Nan,8.3,10.0,verhovina.uk.html,4
5,"зупиняюсь в цьому готелі не перший раз, нажаль...","зупиняюсь в цьому готелі не перший раз, нажаль...","чомусь тапочки вже не дають, чайник є - а паке...",5.4,10.0,verhovina.uk.html,3
6,Достатня кількість побутової техніки для зручн...,Достатня кількість побутової техніки для зручн...,"Фото номеру не відповідають дійсності,оплату з...",3.3,10.0,verhovina.uk.html,2
7,Відносно комфортне проживання за приємну ціну.,"Чистота , тепло та відносно комфортно в номері.",Зламані гачки на вішалці для одягу . Відносно ...,6.7,10.0,verhovina.uk.html,3
8,2 та 5 корпуси умови шикарні,Готель дуже класний у співвідношенні ціна-якіс...,У 3 корпусі давно не робили ремонт та налагоди...,9.6,10.0,verhovina.uk.html,5
9,Хороше співвідношення ціни та якості. За 700 г...,"В цілому, все було добре. Зручне ліжко, вся те...",В день заїзду на вулиці трошки потеплішало і а...,8.8,10.0,verhovina.uk.html,4


In [7]:
def is_review_valid(review):
    if type(review) is not str:
        return False
    try:
        if len(review) == 0:
            return False
        if review == 'Nan':
            return False
        return True
    except:
        return False

In [8]:
def get_pos_neg_reviews(df):
    pos_texts = df['pos_text'].values
    neg_texts = df['neg_text'].values
    
    positives, negatives = [], []
    for i in range(0, len(df)):
        if is_review_valid(pos_texts[i]):
            positives.append(pos_texts[i])
            
        if is_review_valid(neg_texts[i]):
            negatives.append(neg_texts[i])
            
    return positives, negatives

In [9]:
pos_texts, neg_texts = get_pos_neg_reviews(full_df)

In [10]:
stop_words = read_stop_words('data/ukrainian-stopwords.txt')

In [15]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

In [16]:
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [102]:
!ls dataset/trip-advisor/

analyze.ipynb
create-text-classification-dataset.ipynb
trip-advisor-test.csv
trip-advisor-text-classification-test.csv
trip-advisor-text-classification-train.csv
trip-advisor-text-classification-val.csv
trip-advisor-train.csv
trip-advisor-val.csv
tripadvisor-dnipro.csv
tripadvisor-ivano_frankivsk.csv
tripadvisor-kharkiv.csv
tripadvisor-kyiv.csv
tripadvisor-lviv.csv
tripadvisor-odesa.csv
tripadvisor-uzhhorod.csv


In [114]:
trip_advisor_df = pd.read_csv("dataset/trip-advisor/trip-advisor-val.csv")

In [110]:
blum = 'https://www.tripadvisor.ru/Hotel_Review-g295377-d14044085-Reviews-Blum_Hotel-Lviv_Lviv_Oblast.html'

In [115]:
trip_advisor_df.loc[trip_advisor_df['hotel'] == blum]

,hotel,review_ru,review_uk,ratings


In [17]:
booking_df = pd.read_csv('dataset/booking/booking-test.csv')

In [18]:
blum_df = booking_df.loc[booking_df['hotel'] == 'blum.uk.html']

In [19]:
blum_df

,title,pos_text,neg_text,ratingValue,bestRating,hotel,rating
12696,Розташування,Розташування,Брудні і обшарпані стіни номеру Дуже жарко в н...,5.0,10.0,blum.uk.html,2
12697,"Персонал привітний, швидко вирішує усі питання.","Персонал привітний, швидко вирішує усі питання...",Одну з ночей хтось постійно ходив по коридору ...,8.8,10.0,blum.uk.html,4
12698,"Розташування, співвідношення ціни та якості, с...","Розташування, співвідношення ціни та якості, с...","Було б непогано мати чайник в номері, чай та к...",10.0,10.0,blum.uk.html,5
12699,Все супер. Рекомендую,"Знаходиться не далеко від центру, близько всі ...","Запах в номері, якійсь був солодкий, напевно в...",9.2,10.0,blum.uk.html,5
12700,"Хороший готель, схований від шумного центру, т...","Хороший готель, схований від шумного центру, т...","шумоізоляція, та досить велика щілина внизу дв...",9.6,10.0,blum.uk.html,5
12701,Сніданок неперевершений.,Сніданок неперевершений. Необмежена кількість ...,звукоізоляція могла б бути і кращою . Не зайви...,9.2,10.0,blum.uk.html,5
12702,Чудове місце для короткочасних поїздок,"Гарний дизайн готелю, чудове розташування, при...","Сніданок в ціні номеру міг бути більшим, до ка...",8.3,10.0,blum.uk.html,4
12703,Затишний готель у серці Львова;,Затишний готель у серці Львова; привітний нена...,"Сніданок: хотілося б шведський стіл, а не кіль...",8.8,10.0,blum.uk.html,4
12704,наступної мандрівки до Львова оберемо цей готель.,Відмінне розташування готелю! Дуже близько до ...,"один маленький мінус, але нам це не завдало не...",10.0,10.0,blum.uk.html,5
12705,"Сніданок - великі порції, смачно!","Сніданок - великі порції, смачно! 1 страва без...",Сам готель усередині нагадує гуртожиток чи хос...,6.3,10.0,blum.uk.html,3


In [33]:
blum_df['neg_text'].values[11]

'Nan'

In [34]:
neg_texts = [text for text in blum_df['neg_text'].values if text != 'Nan' and len(text) > 0]

In [35]:
neg_texts

['Брудні і обшарпані стіни номеру Дуже жарко в номері Відсутність ліфту Слабий напору води Сейф який нам так і не відкрили, т . б . скористатися ним ми не мали змоги',
 'Одну з ночей хтось постійно ходив по коридору і ми не могли нормально спати через дзвінкий стукіт підборів об плитку . З Комаровим покриттям в коридорі було б комфортніше . Меблі в номері вже трохи втомлені',
 'Було б непогано мати чайник в номері, чай та каву, як це часто буває.',
 'Запах в номері, якійсь був солодкий, напевно від ароматизатор.',
 'шумоізоляція, та досить велика щілина внизу дверей на коридор, коли хтось ходить заважає.. невеликий поріг би виправив це!',
 'звукоізоляція могла б бути і кращою . Не зайвим був би косметичний ремонт номера.',
 "Сніданок в ціні номеру міг бути більшим, до кав'ярні треба йти через вулицю. Надзвичайно тонкі стіни в номерах - чути кожне слово",
 'Сніданок: хотілося б шведський стіл, а не кілька страв на вибір. Дивний душ у номері: важко регулювати температуру води, незручно с

In [79]:
len(neg_texts)

279

### Neagtive TF-IDF

In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [46]:
len(neg_texts)

279

In [73]:
neg_cv=CountVectorizer(max_df=0.85,stop_words=stop_words,max_features=1000, ngram_range=(2,2))
neg_word_count_vector=neg_cv.fit_transform(full_df['neg_text'].values)
neg_word_count_vector.shape

(134083, 1000)

In [74]:
from sklearn.feature_extraction.text import TfidfTransformer

neg_tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
neg_tfidf_transformer.fit(neg_word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [78]:
# get the document that we want to extract keywords from
#doc="Хороше розташування. Безпечна парковка на території готелю. Наявність холодильника та чайника в номері.\nЩоденне прибирання."
#doc  = "Готель дуже зручно розташований. Якщо вам в номері не сидіти то можна спокійно бронювати. Сніданок хороший. Номери стандартні НЕ великі. Персонал ввічливий. Чисто. Після ремонту свіжо. Проблема була з водою, гаряча швидко закінчилася. Але на наступний день з'явилася)). Тапки були одні речі. Мабуть комплектацією дійсно не заморочуються. Але якщо б сказали принесли напевно, ми не просили. Вид з нашого Номери 305 був на вікна житлового будинку, але якщо вийти на балкончик і подивитися на право красиво. Відмінний вид з ресторану на місто. Таксі нам викликали на прощання, доїхали до вокзалу за 3 ціни))))) але зате водій був ввічливий і машина хороша. На фото вид з ресторану."
# doc = neg_texts[123]


#generate tf-idf for the given document
neg_tf_idf_vector=neg_tfidf_transformer.transform(neg_cv.transform(neg_texts))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(neg_tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(neg_cv.get_feature_names(),sorted_items,100)

# now print the results
# print("\n===Review text===")
# print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


===Keywords===
єдиний мінус 1.0
шумоізоляція номерів 1.0
шумоізоляція відсутня 1.0
шведський стіл 1.0
чутно сусідів 1.0
хтось ходить 1.0
тонкі стіни 1.0
таку вартість 1.0
сутність сусідів 1.0
сутність номері 1.0
сусідньому номері 1.0
сподобався сніданок 1.0
сніданок смачний 1.0
слабка шумоізоляція 1.0
скромний сніданок 1.0
підлогу ванній 1.0
потрапити номер 1.0
постільна білизна 1.0
погана шумоізоляція 1.0
погана звукоізоляція 1.0
першому поверсі 1.0
персонал привітний 1.0
останньому поверсі 1.0
об 39 1.0
номері холодно 1.0
номері одне 1.0
номері запах 1.0
номери маленькі 1.0
номер маленький 1.0
номер вікном 1.0
му поверсі 1.0
москітну сітки 1.0
мило шампунь 1.0
меблі номері 1.0
маленький номер 1.0
маленький вибір 1.0
ліжко зручна 1.0
круті сходи 0.766
косметичний ремонт 1.0
зубної щітки 1.0
звукоізоляція чути 1.0
звукоізоляція погана 1.0
звукоізоляція відсутня 1.0
запах номері 1.0
заважає спати 1.0
душовій кабіні 1.0
душова кабінка 1.0
душова кабіна 1.0
деяких місцях 1.0
гаряча вода 

In [ ]:
neg_igrans = [
    'погана шумоізоляція',
    
]

In [92]:
keywords.keys()

dict_keys(['єдиний мінус', 'шумоізоляція номерів', 'шумоізоляція відсутня', 'шведський стіл', 'чутно сусідів', 'хтось ходить', 'тонкі стіни', 'таку вартість', 'сутність сусідів', 'сутність номері', 'сусідньому номері', 'сподобався сніданок', 'сніданок смачний', 'слабка шумоізоляція', 'скромний сніданок', 'підлогу ванній', 'потрапити номер', 'постільна білизна', 'погана шумоізоляція', 'погана звукоізоляція', 'першому поверсі', 'персонал привітний', 'останньому поверсі', 'об 39', 'номері холодно', 'номері одне', 'номері запах', 'номери маленькі', 'номер маленький', 'номер вікном', 'му поверсі', 'москітну сітки', 'мило шампунь', 'меблі номері', 'маленький номер', 'маленький вибір', 'ліжко зручна', 'круті сходи', 'косметичний ремонт', 'зубної щітки', 'звукоізоляція чути', 'звукоізоляція погана', 'звукоізоляція відсутня', 'запах номері', 'заважає спати', 'душовій кабіні', 'душова кабінка', 'душова кабіна', 'деяких місцях', 'гаряча вода', 'відсутність сніданку', 'відсутність звукоізоляції', 

In [96]:
from gensim.models import KeyedVectors
import numpy as np
from nltk.cluster import KMeansClusterer
import nltk

In [93]:
class ClusterNgram:
    def __init__(self, word2vec):
        self.word2vec = word2vec
        
        
    
    def cluster_n_grams(self, ngrams, num_cluster):
        X = self.__convert_ngrams_to_vectors(ngrams)
        kclusterer = KMeansClusterer(num_cluster, 
                                     distance=nltk.cluster.util.cosine_distance, 
                                     repeats=100,
                                     avoid_empty_clusters=True)
        y = kclusterer.cluster(X, assign_clusters=True)
        
        top_indices = self.__find_top_n_gram_indices(X, y, kclusterer.means())
        
        return np.array(ngrams)[top_indices]
        
        
    
    def __find_top_n_gram_indices(self, X, y, centroids):
        min_dis = len(centroids)*[100]
        top_indices = len(centroids)*[-1]
        for i in range(0, len(centroids)):
            for j in range(0, len(X)):
                if y[j] != i:
                    continue
                
                dis = nltk.cluster.util.cosine_distance(X[j], centroids[i])
                if min_dis[i] > dis:
                    min_dis[i] = dis
                    top_indices[i] = j
        
        return top_indices
                    
    
    def __convert_ngrams_to_vectors(self, ngrams):
        X = []
        for ngram in ngrams:
            try:
                words = tokenize_uk.tokenize_words(ngram)
                x = word2vec[words[0]]
                for i in range(1, len(words)):
                    x = x + word2vec[words[i]]
                X.append(x)
            except:
                X.append(np.zeros(300))
        return X

In [94]:
embeddings_file = "./embeddings/ubercorpus.cased.tokenized.word2vec.300d"

In [97]:
word2vec = KeyedVectors.load_word2vec_format(embeddings_file)

In [99]:
cluster_ngram = ClusterNgram(word2vec)

In [101]:
%time result = cluster_ngram.cluster_n_grams(list(keywords.keys()), 10)

KeyboardInterrupt: 

In [51]:
pos_texts = [text for text in blum_df['pos_text'].values if text != 'Nan' and len(text) > 0]

### Positive TF-IDF

In [52]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [53]:
len(pos_texts)

357

In [64]:
pos_cv=CountVectorizer(max_df=0.85,stop_words=stop_words,max_features=1000, ngram_range=(2,2))
pos_word_count_vector=pos_cv.fit_transform(full_df['pos_text'].values)
pos_word_count_vector.shape

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(134083, 1000)

In [65]:
len(pos_cv.vocabulary_)

1000

In [66]:
from sklearn.feature_extraction.text import TfidfTransformer

pos_tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
pos_tfidf_transformer.fit(pos_word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [67]:
pos_word_count_vector

<134083x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 298443 stored elements in Compressed Sparse Row format>

In [68]:
# you only needs to do this once
pos_feature_names=pos_cv.get_feature_names()

In [69]:
pos_feature_names

['10 10',
 '10 15',
 '10 хв',
 '10 хвилин',
 '10 хвилинах',
 '12 00',
 '14 00',
 '15 20',
 '15 хв',
 '15 хвилин',
 '15 хвилинах',
 '20 хв',
 '20 хвилин',
 '23 00',
 '30 хвилин',
 '39 лякливий',
 '39 язаний',
 '39 язка',
 '39 язково',
 '39 язку',
 '39 язно',
 '39 яке',
 '39 яти',
 '39 ятки',
 '39 яток',
 '39 ять',
 '39 єр',
 '39 єру',
 '39 єю',
 '39 їзд',
 '39 їзду',
 'quota quota',
 'wi fi',
 'адекватна ціна',
 'аеропорту жуляни',
 'акуратний номер',
 'безкоштовна парковка',
 'березі моря',
 'близьке розташування',
 'близькість метро',
 'близькість моря',
 'близькість центру',
 'білизна рушники',
 'білизна чиста',
 'білосніжна постіль',
 'білосніжні рушники',
 'біля вокзалу',
 'біля входу',
 'біля готелю',
 'біля метро',
 'біля моря',
 'вай фай',
 'ванна кімната',
 'ванною кімнатою',
 'ванні приналежності',
 'ванній кімнаті',
 'вартість номера',
 'вартість проживання',
 'ввічливий персонал',
 'ввічливий привітний',
 'ввічливий уважний',
 'ввічливий чуйний',
 'вдале розташування',
 'вел

In [77]:


# get the document that we want to extract keywords from
#doc="Хороше розташування. Безпечна парковка на території готелю. Наявність холодильника та чайника в номері.\nЩоденне прибирання."
#doc  = "Готель дуже зручно розташований. Якщо вам в номері не сидіти то можна спокійно бронювати. Сніданок хороший. Номери стандартні НЕ великі. Персонал ввічливий. Чисто. Після ремонту свіжо. Проблема була з водою, гаряча швидко закінчилася. Але на наступний день з'явилася)). Тапки були одні речі. Мабуть комплектацією дійсно не заморочуються. Але якщо б сказали принесли напевно, ми не просили. Вид з нашого Номери 305 був на вікна житлового будинку, але якщо вийти на балкончик і подивитися на право красиво. Відмінний вид з ресторану на місто. Таксі нам викликали на прощання, доїхали до вокзалу за 3 ціни))))) але зате водій був ввічливий і машина хороша. На фото вид з ресторану."
doc = neg_texts[123]


#generate tf-idf for the given document
pos_tf_idf_vector=pos_tfidf_transformer.transform(pos_cv.transform(pos_texts))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(pos_tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(pos_feature_names,sorted_items,100)

# now print the results
# print("\n===Review text===")
# print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


===Keywords===
чудовий персонал 1.0
чудовий готель 1.0
чистота персонал 1.0
чистота затишок 1.0
чисто тихо 1.0
чисто персонал 0.81
чисто охайно 1.0
чисто комфортно 1.0
ціна якість 1.0
центрі міста 1.0
хороший сніданок 1.0
хороший персонал 1.0
уважний персонал 1.0
тиша спокій 1.0
супер розташування 1.0
ставлення персоналу 1.0
сніданок смачний 1.0
сніданок різноманітний 1.0
сніданок номер 1.0
сніданок вибір 1.0
смачні сніданки 1.0
свіжим ремонтом 1.0
розташування чудове 1.0
розташування чистота 0.762
розташування персонал 0.796
розташування готелю 0.783
розташування відмінне 1.0
розташований центрі 1.0
прямо центрі 1.0
приємний персонал 1.0
привітний персонал 1.0
привабливий персонал 1.0
питна вода 1.0
персонал чистота 1.0
персонал уважний 1.0
персонал ввічливий 1.0
номері чисто 1.0
місце розташування 1.0
кава чай 0.827
зупинятися готелі 1.0
зручно чисто 1.0
зручне розташування 1.0
затишний номер 1.0
затишне місце 1.0
доброзичливий персонал 1.0
готель чистий 1.0
готель центрі 1.0
готель

In [71]:
import pickle

In [115]:
pickle.dump(pos_tfidf_transformer, open("pos_tfidf_transformer.pickle", "wb"))

In [116]:
pickle.dump(pos_cv, open("pos_cv.pickle", "wb"))

In [ ]:
# open a file, where you stored the pickled data
file = open('important', 'rb')

# dump information to that file
data = pickle.load(file)

# close the file
file.close()

In [ ]:
pos_cv

In [117]:
neg_text

NameError: name 'neg_text' is not defined

In [75]:
pos_texts[123]

'Хороше розташування. Безпечна парковка на території готелю. Наявність холодильника та чайника в номері.\nЩоденне прибирання.'

In [119]:
neg_texts[123]

'Головне не переплутати з іншим готелем "Верховина", котрий знаходиться на відстані 2,5 км. При поселенні потрібно пройти квест "знайди рецепцію, котра знаходиться в корпусі №Х, а потім звідти іди на поселення в корпус №У"'

## Read dataset and create docs

In [9]:
full_df['title'].iloc[0]

'Лише дівчата на рецепції - три рази мені міняли кімнату.'

In [12]:
title = literal_eval(full_df.iloc[0]['title'])

SyntaxError: invalid syntax (<unknown>, line 1)

In [10]:
' '.join(word for word in title)

NameError: name 'title' is not defined

In [15]:
def create_docs(data):
    docs = []
    for i in range(0, len(data)):
        text = ""
        
        title = data.iloc[i]['title']
        if len(title) > 0 and title[0] != 'Nan':
            text += title
            
        pos = data.iloc[i]['pos_text']
        if len(pos) > 0 and pos[0] != 'Nan':
            text += ' '
            text += pos
            
        neg = data.iloc[i]['neg_text']
        if len(neg) > 0 and neg[0] != 'Nan':
            text += ' '
            text += neg
            
        docs.append(text)
    return docs
        

In [16]:
docs = create_docs(full_df)

### Create the IDF

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
stop_words = read_stop_words('data/ukrainian-stopwords.txt')

In [22]:
cv=CountVectorizer(max_df=0.9,stop_words=stop_words)
word_count_vector=cv.fit_transform(docs)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [23]:
word_count_vector.shape

(134083, 91583)

Let's limit our vocabulary size to 10,000

In [24]:
cv=CountVectorizer(max_df=0.85,stop_words=stop_words,max_features=1000)
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

(134083, 1000)

### TfidfTransformer to Compute Inverse Document Frequency (IDF) 

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

Let's look at some of the IDF values:

In [26]:
tfidf_transformer.idf_

array([4.88795706, 4.41686193, 5.77095315, 6.0609854 , 5.69715961,
       6.02289655, 4.98057702, 5.14846648, 6.34007702, 6.67499526,
       6.56789712, 5.2843625 , 3.19101645, 6.35417279, 6.04812724,
       7.42632439, 6.32464462, 5.10050892, 6.75884957, 1.95952893,
       4.38884889, 7.27283226, 5.10005878, 5.83455314, 5.23629409,
       5.67292579, 4.9254174 , 6.00716589, 6.52022365, 6.48905706,
       6.63035448, 6.08479605, 6.8555792 , 5.65630491, 5.50439941,
       6.7059028 , 5.72535385, 6.51465261, 5.65630491, 6.29447642,
       6.51650618, 6.59161365, 5.33201694, 6.59361565, 6.48186279,
       5.74403012, 6.62413684, 5.99277715, 6.24377765, 5.97319002,
       5.45442188, 5.95503682, 5.84303176, 6.35259675, 6.54664028,
       6.30944676, 5.93098966, 6.08359195, 5.97426818, 6.38459948,
       5.97426818, 5.66894331, 6.03543232, 5.40011837, 6.23254158,
       6.57181102, 5.78335366, 6.6663372 , 6.63243564, 6.55817887,
       4.83375573, 6.24377765, 4.44802429, 5.6361022 , 5.99939

### Computing TF-IDF and Extracting Keywords

In [27]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [34]:
# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs[0]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# now print the results
print("\n===Review text===")
print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


===Review text===
Лише дівчата на рецепції - три рази мені міняли кімнату. Лише дівчата на рецепції - три рази мені міняли кімнату. Дякую їм. Все. Одного досвіду вистарчило, щоб більше сюди не повертатися. Готель лише для гостей, яким байдуже умови , чистота і комфорт.

===Keywords===
рази 0.472
дівчата 0.458
кімнату 0.442
рецепції 0.4
сюди 0.221
умови 0.218
гостей 0.207
комфорт 0.204
чистота 0.145
готель 0.098


In [29]:
a = tf_idf_vector=tfidf_transformer.transform(cv.transform(docs[:1]))

In [30]:
cv.transform(docs[:2])

<2x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 22 stored elements in Compressed Sparse Row format>

In [31]:
sorted_items=sort_coo(a.tocoo())

In [32]:
sorted_items

[(714, 0.47177032223963566),
 (283, 0.4583165725717789),
 (422, 0.4423654769281214),
 (739, 0.39990782054595764),
 (838, 0.22144445972314214),
 (875, 0.2180827599072353),
 (210, 0.20699220385566333),
 (390, 0.20439509883082757),
 (936, 0.1447041801699384),
 (215, 0.09800527891767899)]